In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import pickle
from shapely import LineString,MultiLineString
from shapely import geometry, ops
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

C:\Users\Diametrix\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Diametrix\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Diametrix\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\Diametrix\AppData\Roaming\Python\Python39\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\1400089426.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force 

In [2]:
iris_data=pd.read_excel(r"Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - IRIS\\Geodata_IRIS_essentiels.xlsx")
contour_iris=gpd.read_file(r'Y:\REFERENTIEL DATA\RP 2022 FRANCE\CARTOGRAPHIE\Cartographie_France_IRIS_2019\CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019\CONTOURS-IRIS.shp').to_crs("3857")

In [3]:
path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm filtre\\"

In [4]:
path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm filtre\\"
list_departement=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Liste-Excel-des-departements-francais.xlsx")

In [5]:
indicateurs=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Données_OSM_métie1.xlsx",sheet_name='shops')
indicateurs=indicateurs[indicateurs['à garder']==1]
indicateurs=indicateurs.set_index("shop")

In [6]:
indicateurs_amenity=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Données_OSM_métie1.xlsx",sheet_name='amenity')
indicateurs_amenity=indicateurs_amenity[indicateurs_amenity['à garder']==1]
indicateurs_amenity=indicateurs_amenity.set_index('amenity')

In [7]:
indicateurs_loisire=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Données_OSM_métie1.xlsx",sheet_name='leisure')
indicateurs_loisire=indicateurs_loisire[indicateurs_loisire['à garder']==1]
indicateurs_loisire=indicateurs_loisire.set_index('loisire')

In [8]:
indicateurs_amenity['attracteur'].value_counts()

Culture loisir -amenity                    20
Mobilité                                   20
pt d'intérert                              13
tous -amenity                              11
Soin de la personne et Médical -amenity    10
Service public                              9
restauration                                8
Name: attracteur, dtype: int64

In [9]:
columns_socio=['C17_MEN',
       'C17_MEN_CS1', 'C17_MEN_CS2', 'C17_MEN_CS3', 'C17_MEN_CS4',
       'C17_MEN_CS5', 'C17_MEN_CS6', 'C17_MEN_CS7', 'C17_MEN_CS8', 'P17_POP',
       'P17_POP0002', 'P17_POP0305', 'P17_POP0610', 'P17_POP1117',
       'P17_POP1824', 'P17_POP2539', 'P17_POP4054', 'P17_POP5564',
       'P17_POP6579', 'P17_POP80P', 'P17_POP15P']

In [10]:
data_gsa=pd.read_csv(r"Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - Reseaux de distibution\GSA_GSS 2020"+r'\GSA_2022_clean.csv')
data_gsa = gpd.GeoDataFrame(
    data_gsa, geometry=gpd.points_from_xy(data_gsa.X__WGS84_, data_gsa.Y__WGS84_)).set_crs("EPSG:4326").to_crs("3857")

C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\294234986.py:1: DtypeWarning: Columns (9,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data_gsa=pd.read_csv(r"Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - Reseaux de distibution\GSA_GSS 2020"+r'\GSA_2022_clean.csv')


In [11]:
data_gsa.CP=data_gsa.CP.apply(lambda x:str(x) if len(str(x)) ==5 else  "0"+str(x))

In [12]:
indicateurs['secteurs'].value_counts()

Equipement maison      40
Alimentaire            24
Culture Loisir         21
Equipement Personne    18
Soin Personne          10
Medical                 4
GSA                     4
Sport                   2
Name: secteurs, dtype: int64

In [61]:
#fpour le loisir
def f(x):

    if x['leisure'] in indicateurs_loisire.index:
        return indicateurs_loisire.loc[x['leisure']]['attracteur'] 
    else :
        return "Attracteur pour tous -loisire"

#g pour le secteur d'activité des magasins
def g(x):
    if x['shop'] in indicateurs.index:
        return indicateurs.loc[x['shop']]['secteurs'] 
    else :
        return "Autre"
    



def get_KPI(row,batiments,shops,gsa_comp,gss_comp,loisire,tourism,public_transport,amenity):
    
    if "Avenue" in row['name'] or "Quai" in row['name'] or "Boulevard" in row['name']:
        max_distance=75
    else:max_distance=30
        
    socio_demo=batiments[batiments.intersects(row['geometry'].buffer(max_distance))][columns_socio]
    #sauvegarder les batimenets affectés à un segement
    batiments_affectes.update(list(socio_demo.index))
   
    socio_demo_index=list(socio_demo.index)
    #agregation
    socio_demo=socio_demo.sum()
    
    
    #tourism
    tourism_seg=tourism[tourism.intersects(row['geometry'].buffer(max_distance))]
    #sauvegarder les "tourisme" affectés à un segement
    tourism_affectes.update(list(tourism_seg.index))
    
    #loisir
    loisire_seg=loisire[loisire.intersects(row['geometry'].buffer(max_distance))]
    loisire_seg["attracteur"]=loisire_seg.apply(lambda x:f(x),axis=1)
    #sauvegarder les "loisire" affectés à un segement
    loisire_affectes.update(list(loisire_seg.index))
    
 
    #transport 
    public_transport_seg=public_transport[public_transport.intersects(row['geometry'].buffer(max_distance))]
    try:
        metro=public_transport_seg[public_transport_seg['station']=="subway"].drop_duplicates('name')
    except:
        metro=[]
    try:
        tram=public_transport_seg[public_transport_seg['railway']=="tram_stop"].drop_duplicates('name')
    except:
        tram=[]
    reste_transport=len(public_transport_seg)-len(metro)-len(tram)
    #sauvegarder les "transport" affectés à un segement
    public_transport_affectes.update(list(public_transport_seg.index))
    
    #info shops
    shop_seg=shops[shops.intersects(row['geometry'].buffer(max_distance))]
    #sauvegarder les "shops" affectés à un segement
    shop_affectes.update(list(shop_seg.index))

    shop_seg=shop_seg[shop_seg['shop'].isin(indicateurs.index)]
    shop_seg['secteur']=shop_seg.apply(lambda x:g(x),axis=1)
    

    #### gsa_comp_info  
    gsa_comp_info=gsa_comp[gsa_comp.intersects(row['geometry'].buffer(max_distance))]
    #sauvegarder les "gsa" affectés à un segement
    gsa_affectes.update(list(gsa_comp_info.index))
    
    #### gss_comp_info
    gss_comp_info=gss_comp[gss_comp.intersects(row['geometry'].buffer(max_distance))]
    gss_affectes.update(list(gss_comp_info.index))
    
    #amenity
    amenity_seg=amenity[amenity.intersects(row['geometry'].buffer(max_distance))]
    amenity_affectes.update(list(amenity_seg.index))
    amenity_seg=amenity_seg[amenity_seg['amenity'].isin(indicateurs_amenity.index)]
    amenity_seg['attracteur']=indicateurs_amenity.loc[amenity_seg['amenity'].values]['attracteur'].values
    
    
    
    output=socio_demo.to_dict()
    output["Attracteur_tourisme"]=len(tourism_seg)
    output=output|loisire_seg.groupby("attracteur").size().to_dict()
    output["metro"]=len(metro)
    output["tram"]=len(tram)
    output["transport"]=reste_transport
    output=output|shop_seg.groupby("secteur").size().to_dict()
    output=output|amenity_seg.groupby("attracteur").size().to_dict()
    output=output|row.to_dict()
    
    if "GSA" in output.keys():output['GSA']+=len(gsa_comp_info)
    else:output['GSA']=len(gsa_comp_info)
        
    
    if "Sport" in output.keys():output['Sport']+=len(gss_comp_info[gss_comp_info['Type_Mag']=="Sport"])
    else:output['Sport']=len(gss_comp_info[gss_comp_info['Type_Mag']=="Sport"])
    
    
    if "Equipement maison" in output.keys():output['Equipement maison']+=len(gss_comp_info[gss_comp_info['Type_Mag']=="Négoce"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Discount Bazar"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Electrodomestique"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="GMS Bricolage"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Jardinerie"])
    else:output['Equipement maison']=len(gss_comp_info[gss_comp_info['Type_Mag']=="Négoce"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Discount Bazar"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Electrodomestique"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="GMS Bricolage"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Jardinerie"])
     
    
    if "Equipement Personne" in output.keys():output['Equipement Personne']+=len(gss_comp_info[gss_comp_info['Type_Mag']=="GMS Textile Habillement"])
    else:output['Equipement Personne']=len(gss_comp_info[gss_comp_info['Type_Mag']=="GMS Textile Habillement"])
        
    if "Culture Loisir" in output.keys():output["Culture Loisir"]+=len(gss_comp_info[gss_comp_info['Type_Mag']=="Jouets"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Culture Loisirs"])
    else:output["Culture Loisir"]=len(gss_comp_info[gss_comp_info['Type_Mag']=="Jouets"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Culture Loisirs"])
    
    return output ,socio_demo_index,list(tourism_seg.index),list(loisire_seg.index),list(public_transport_seg.index),list(shop_seg.index),list(amenity_seg.index),list(gsa_comp_info.index),list(gss_comp_info.index)
    
    

In [62]:
def affectation_bat(building,batiments_affectes,highways,result,socio_demo_index):
    #selectionner les batiments non affectés à un segement
    batiments_non_aff=building.loc[list(building.index.difference(list(batiments_affectes)))]
    if len(batiments_non_aff)<=0: return socio_demo_index,result
    #centre du batiment
    centre=batiments_non_aff.centroid
    #l'indice (position) du segment le plus proche à un batiment
    index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

    batiments_non_aff['seg']=list(index)
    a=batiments_non_aff.groupby("seg")[columns_socio].sum()
    result.loc[highways.iloc[a.index].index,columns_socio]=result.iloc[a.index][columns_socio].values+a.values

    #l'indice reel du segement associé à ce batiment
    batiments_non_aff['seg_index']=list(highways.iloc[batiments_non_aff.seg].index)
    #rendre l'indice du batiment une colonne
    batiments_non_aff=batiments_non_aff.reset_index()
    batiments_non_aff['bat_index']=batiments_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
    #regrouper les batiments dans une list par segement 
    batiments_non_aff=batiments_non_aff[["bat_index","seg_index"]].groupby('seg_index')['bat_index'].apply(list)
    socio_demo_index.loc[batiments_non_aff.index,'indices']+=pd.DataFrame(batiments_non_aff)['bat_index']
    return socio_demo_index,result

In [63]:
def affectation_loisire(loisire,loisire_affectes,highways,result,loisire_seg_index):
    #centroid des "loisire" non affectés 
    loisire_non_aff=loisire.loc[list(loisire.index.difference(list(loisire_affectes)))]
    if len(loisire_non_aff)<=0: return loisire_seg_index,result
    centre=loisire_non_aff.centroid
    index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

    loisire_non_aff['seg']=list(index)
    loisire_non_aff["attracteur"]=loisire_non_aff.apply(lambda x:f(x),axis=1)
    a=loisire_non_aff.groupby(["seg","attracteur"]).size().reset_index().pivot("seg",'attracteur',0).fillna(0)
    if "Culture loisir -loisire" not in a.columns:a["Culture loisir -loisire"]=[0]*len(a)
    if "Sport -loisire" not in a.columns:a["Sport -loisire"]=[0]*len(a)
    if "Attracteur pour tous -loisire" not in a.columns:a["Attracteur pour tous -loisire"]=[0]*len(a)

    result.loc[highways.iloc[a.index].index,a.columns]=result.iloc[a.index][a.columns].values+a.values


    #l'indice reel du segement associé à ce batiment
    loisire_non_aff['seg_index']=list(highways.iloc[loisire_non_aff.seg].index)
    #rendre l'indice du batiment une colonne
    loisire_non_aff=loisire_non_aff.reset_index()
    loisire_non_aff['loisire_index']=loisire_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
    #regrouper les batiments dans une list par segement 
    loisire_non_aff=loisire_non_aff[["loisire_index","seg_index"]].groupby('seg_index')['loisire_index'].apply(list)
    loisire_seg_index.loc[loisire_non_aff.index,'indices']+=pd.DataFrame(loisire_non_aff)['loisire_index']
    return loisire_seg_index,result

In [64]:
def affectation_tourisme(tourisme,tourism_affectes,highways,result,tourism_seg_index):
    #centroid des "tourisme" non affectés 
    tourisme_non_aff=tourisme.loc[list(tourisme.index.difference(list(tourism_affectes)))]
    if len(tourisme_non_aff)<=0: return tourism_seg_index,result
    centre=tourisme_non_aff.centroid
    index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

    tourisme_non_aff['seg']=list(index)
    a=tourisme_non_aff.groupby("seg").size()
    result.loc[highways.iloc[a.index].index,"Attracteur_tourisme"]=result.iloc[a.index]["Attracteur_tourisme"].values+a.values


    #l'indice reel du segement associé à ce batiment
    tourisme_non_aff['seg_index']=list(highways.iloc[tourisme_non_aff.seg].index)
    #rendre l'indice du batiment une colonne
    tourisme_non_aff=tourisme_non_aff.reset_index()
    tourisme_non_aff['tourisme_index']=tourisme_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
    #regrouper les batiments dans une list par segement 
    tourisme_non_aff=tourisme_non_aff[["tourisme_index","seg_index"]].groupby('seg_index')['tourisme_index'].apply(list)
    tourism_seg_index.loc[tourisme_non_aff.index,'indices']+=pd.DataFrame(tourisme_non_aff)['tourisme_index']
    
    return tourism_seg_index,result

In [65]:
def affectation_shops(shops,shop_affectes,highways,result,shop_seg_index):
    #centroid des "shops" non affectés 
    shop_non_aff=shops.loc[list(shops.index.difference(list(shop_affectes)))]
    if len(shop_non_aff)<=0: return shop_seg_index,result
    centre=shop_non_aff.centroid
    index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

    shop_non_aff['seg']=list(index)
    shop_non_aff=shop_non_aff[shop_non_aff['shop'].isin(indicateurs.index)]
    shop_non_aff['secteur']=shop_non_aff.apply(lambda x:g(x),axis=1)

    a=shop_non_aff.groupby(["seg","secteur"]).size().reset_index().pivot("seg",'secteur',0).fillna(0)
    
    if "Equipement maison" not in a.columns:a["Equipement maison"]=[0]*len(a)
    if "Alimentaire" not in a.columns:a["Alimentaire"]=[0]*len(a)
    if "Culture Loisir" not in a.columns:a["Culture Loisir"]=[0]*len(a)
    if "Equipement Personne" not in a.columns:a["Equipement Personne"]=[0]*len(a)
    if "Soin Personne" not in a.columns:a["Soin Personne"]=[0]*len(a)
    if "Medical" not in a.columns:a["Medical"]=[0]*len(a)
    if "GSA" not in a.columns:a["GSA"]=[0]*len(a)
    if "Sport" not in a.columns:a["Sport"]=[0]*len(a)

    
    
    result.loc[highways.iloc[a.index].index,a.columns]=result.iloc[a.index][a.columns].values+a.values

    #l'indice reel du segement associé à ce batiment
    shop_non_aff['seg_index']=list(highways.iloc[shop_non_aff.seg].index)
    #rendre l'indice du batiment une colonne
    shop_non_aff=shop_non_aff.reset_index()
    shop_non_aff['shop_index']=shop_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
    #regrouper les batiments dans une list par segement 
    shop_non_aff=shop_non_aff[["shop_index","seg_index"]].groupby('seg_index')['shop_index'].apply(list)
    shop_seg_index.loc[shop_non_aff.index,'indices']+=pd.DataFrame(shop_non_aff)['shop_index']
    
    return shop_seg_index,result

In [66]:
def affectation_amenity(amenity,amenity_affectes,highways,result,amenity_seg_index):
    #centroid des "amenity" non affectés 
    amenity_non_aff=amenity.loc[list(amenity.index.difference(list(amenity_affectes)))]
    if len(amenity_non_aff)<=0: return amenity_seg_index,result
    centre=amenity_non_aff.centroid
    index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

    amenity_non_aff['seg']=list(index)
    amenity_non_aff=amenity_non_aff[amenity_non_aff['amenity'].isin(indicateurs_amenity.index)]
    amenity_non_aff['attracteur']=indicateurs_amenity.loc[amenity_non_aff['amenity'].values]['attracteur'].values
    a=amenity_non_aff.groupby(['seg','attracteur']).size().reset_index().pivot("seg",'attracteur',0).fillna(0)
    
    if "Culture loisir -amenity" not in a.columns:a["Culture loisir -amenity"]=[0]*len(a)
    if "Mobilité" not in a.columns:a["Mobilité"]=[0]*len(a)
    if "pt d'intérert" not in a.columns:a["pt d'intérert"]=[0]*len(a)
    if "tous -amenity" not in a.columns:a["tous -amenity"]=[0]*len(a)
    if "Soin de la personne et Médical -amenity" not in a.columns:a["Soin de la personne et Médical -amenity"]=[0]*len(a)
    if "Service public" not in a.columns:a["Service public"]=[0]*len(a)
    if "restauration" not in a.columns:a["restauration"]=[0]*len(a)

    result.loc[highways.iloc[a.index].index,a.columns]=result.iloc[a.index][a.columns].values+a.values


    #l'indice reel du segement associé à ce batiment
    amenity_non_aff['seg_index']=list(highways.iloc[amenity_non_aff.seg].index)
    #rendre l'indice du batiment une colonne
    amenity_non_aff=amenity_non_aff.reset_index()
    amenity_non_aff['amenity_index']=amenity_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
    #regrouper les batiments dans une list par segement 
    amenity_non_aff=amenity_non_aff[["amenity_index","seg_index"]].groupby('seg_index')['amenity_index'].apply(list)
    amenity_seg_index.loc[amenity_non_aff.index,'indices']+=pd.DataFrame(amenity_non_aff)['amenity_index']
    return amenity_seg_index,result


In [76]:
def affectation_gsa(gsa_comp,gsa_affectes,highways,result,gsa_seg_index):

    #centroid des "gsa" non affectés 
    gsa_non_aff=gsa_comp.loc[list(gsa_comp.index.difference(list(gsa_affectes)))]
    if len(gsa_non_aff)<=0: return gsa_seg_index,result
    centre=gsa_non_aff.centroid
    
    index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]
  
    gsa_non_aff['seg']=list(index)
    a=gsa_non_aff.groupby('seg').size()
    
    result.loc[highways.iloc[a.index].index,"GSA"]+=a.values
    #l'indice reel du segement associé à ce batiment
    gsa_non_aff['seg_index']=list(highways.iloc[gsa_non_aff.seg].index)
    gsa_non_aff=gsa_non_aff.reset_index()
    gsa_non_aff=gsa_non_aff[["index","seg_index"]].groupby('seg_index')['index'].apply(list)

    gsa_seg_index.loc[gsa_non_aff.index,'indices']+=pd.DataFrame(gsa_non_aff)['index']
    return gsa_seg_index,result

In [77]:
def affectation_gss(gss_comp,gss_affectes,highways,result,gss_seg_index):
    #centroid des "gss" non affectés 
    gss_non_aff=gss_comp.loc[list(gss_comp.index.difference(list(gss_affectes)))]
    if len(gss_non_aff)<=0: return gss_seg_index,result
    centre=gss_non_aff.centroid
    index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

    gss_non_aff['seg']=list(index)
    #else:output['Equipement maison']=len(gss_comp_info[gss_comp_info['Type_Mag']=="Jardinerie"])
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Négoce","Equipement maison")
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Discount Bazar","Equipement maison")
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Electrodomestique","Equipement maison")
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("GMS Bricolage","Equipement maison")
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Jardinerie","Equipement maison")
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("LISA","Equipement maison")
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("GMS Textile Habillement","Equipement Personne")
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Jouets","Culture Loisir")  
    gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Culture Loisirs","Culture Loisir")  

    a=gss_non_aff.groupby(['seg','Type_Mag']).size().reset_index().pivot("seg",'Type_Mag',0).fillna(0)  

    if "Equipement maison" not in a.columns:a["Equipement maison"]=[0]*len(a)
    if "Culture Loisir" not in a.columns:a["Culture Loisir"]=[0]*len(a)
    if "Equipement Personne" not in a.columns:a["Equipement Personne"]=[0]*len(a)

    
    
    result.loc[highways.iloc[a.index].index,a.columns]=result.iloc[a.index][a.columns].values+a.values
    #l'indice reel du segement associé à ce batiment
    gss_non_aff['seg_index']=list(highways.iloc[gss_non_aff.seg].index)
    gss_non_aff=gss_non_aff.reset_index()
    gss_non_aff=gss_non_aff[["index","seg_index"]].groupby('seg_index')['index'].apply(list)
    gss_seg_index.loc[gss_non_aff.index,'indices']+=pd.DataFrame(gss_non_aff)['index']
    
    return gss_seg_index,result

In [78]:
def affectation_public_transport(public_transport,public_transport_affectes,highways,result,public_transport_seg_index):
    
    #centroid des "public_transport_affectes" non affectés 
    public_transport_non_aff=public_transport.loc[list(public_transport.index.difference(list(public_transport_affectes)))]
    if len(public_transport_non_aff)<=0: return public_transport_seg_index,result
    centre=public_transport_non_aff.centroid
    index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

    public_transport_non_aff['seg']=list(index)

    if "subway" in public_transport_non_aff.columns:
        metro=public_transport_non_aff[~public_transport_non_aff['subway'].isna()].drop_duplicates('name').groupby("seg").size()
    else:
        metro=pd.Series(0,index=public_transport_non_aff['seg'].unique())
       

    if "railway" in public_transport_non_aff.columns:
        tram=public_transport_non_aff[public_transport_non_aff['railway']=="tram_stop"].drop_duplicates('name').groupby("seg").size()
      
    else:
        tram=pd.Series(0,index=public_transport_non_aff['seg'].unique())
    
    transport=public_transport_non_aff.groupby("seg").size()
    transport.loc[metro.index]-=metro.values
    transport.loc[tram.index]-=tram.values
   
   
    result.loc[highways.iloc[metro.index].index,"metro"]=result.iloc[metro.index]["metro"].values+metro.values
    result.loc[highways.iloc[tram.index].index,"tram"]=result.iloc[tram.index]["tram"].values+tram.values
    result.loc[highways.iloc[transport.index].index,"transport"]=result.iloc[transport.index]["transport"].values+transport.values

    #l'indice reel du segement associé à ce batiment
    public_transport_non_aff['seg_index']=list(highways.iloc[public_transport_non_aff.seg].index)
 
    #rendre l'indice du transport une colonne
    public_transport_non_aff=public_transport_non_aff.reset_index()
    
    public_transport_non_aff['transport_index']=public_transport_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)

    #regrouper les batiments dans une list par segement 
    public_transport_non_aff=public_transport_non_aff[["transport_index","seg_index"]].groupby('seg_index')['transport_index'].apply(list)
    
    if len(public_transport_non_aff)>0:
        public_transport_seg_index.loc[public_transport_non_aff.index,'indices']+=pd.DataFrame(public_transport_non_aff)['transport_index']
    return public_transport_seg_index,result

In [80]:
final_result=None
for j ,row in list_departement.iloc[59:].iterrows():
        batiments_affectes=set()
        tourism_affectes=set()
        loisire_affectes=set()

        public_transport_affectes=set()
        shop_affectes=set()
        amenity_affectes=set()
        gsa_affectes=set()
        gss_affectes=set()

        try:
            place_name=row['Département']
            print(place_name)
            region=row['Région']
            building=pd.read_pickle(path+r"batiments_est_pop\\"+row['Département'].lower()+".pkl").to_crs('EPSG:3857')
            building=building.fillna(0)
            highways=pd.read_pickle(path+r"rue\\"+row['Département'].lower()+"_highways.pkl").to_crs('EPSG:3857')
            
            highways=highways.loc[~highways.index.duplicated(keep="first")]
           
            shops=pd.read_pickle(path+r"magasin\\"+row['Département'].lower()+"_shops.pkl").to_crs('EPSG:3857')
           
        except:
            
            print("error")
            continue
            
        try:
            gsa=pd.read_pickle(path+r"GSA\\"+row['Département'].lower()+"_gsa.pkl").to_crs('EPSG:3857')
        except:
            gsa=None
        try:
            gss=pd.read_pickle(path+r"GSS\\"+row['Département'].lower()+"_gss.pkl").to_crs('EPSG:3857')
        except:
            gss=None
        

        
        loisire=pd.read_pickle(path+r"loisire\\"+row['Département'].lower()+"_leisure.pkl").to_crs('EPSG:3857')
        tourism=pd.read_pickle(path+r"tourisme\\"+row['Département'].lower()+"_tourism.pkl").to_crs('EPSG:3857')
        public_transport=pd.read_pickle(path+r"public_transports\\"+row['Département'].lower()+".pkl").to_crs('EPSG:3857')
        amenity=pd.read_pickle(path+r"amenity\\"+row['Département'].lower()+"_amenity.pkl").to_crs('EPSG:3857')
        
        
        
       
       
       
        output=highways.apply(lambda x:get_KPI(x,building,shops,gsa,gss,loisire,tourism,public_transport,amenity),axis=1)
        result=pd.DataFrame.from_dict([output.values[i][0] for i in range(len(output))]).set_index(output.index)
 
        #gsa data
        data_gsa_sep=data_gsa[data_gsa.CP.apply(lambda x:x[0:2])==str(row['N°'])]
        gsa_aff=[]
        for i in range(len(data_gsa_sep)):
            distance=highways.distance(data_gsa_sep.iloc[i]['geometry'])
            if distance.min()<=1000:
                gsa_aff.append(distance.argmin())
            else:
                gsa_aff.append(None)
        
        data_gsa_sep['highway']=gsa_aff
        surface=data_gsa_sep.groupby("highway")["Surface_de_vente"].sum()

        indices=result.iloc[surface.index].index
        result.loc[indices,'Surface_de_vente_GSA']=surface.values
        
     
        if "Culture loisir -loisire" not in result.columns:result["Culture loisir -loisire"]=[0]*len(result)
        if "Sport -loisire" not in result.columns:result["Sport -loisire"]=[0]*len(result)
        if "Attracteur pour tous -loisire" not in result.columns:result["Attracteur pour tous -loisire"]=[0]*len(result)
        if "Attracteur_tourisme"  not in result.columns:result["Attracteur_tourisme"]=[0]*len(result)
        if "metro"  not in result.columns:result["metro"]=[0]*len(result)
        if "tram"  not in result.columns:result["tram"]=[0]*len(result)
        if "transport"  not in result.columns:result["transport"]=[0]*len(result)
        if "GSA" not in result.columns:result["GSA"]=[0]*len(result)

        result=result.fillna(0)
       
        #les elements non affectés
        ## pour chaque segement de rue on associé ses elements dans l'approximité
        socio_demo_index=pd.DataFrame({"indices":[output.values[i][1] for i in range(len(output))]}).set_index(output.index)
        tourism_seg_index=pd.DataFrame({"indices":[output.values[i][2] for i in range(len(output))]}).set_index(output.index)
        loisire_seg_index=pd.DataFrame({"indices":[output.values[i][3] for i in range(len(output))]}).set_index(output.index)
        public_transport_seg_index=pd.DataFrame({"indices":[output.values[i][4] for i in range(len(output))]}).set_index(output.index)
        shop_seg_index=pd.DataFrame({"indices":[output.values[i][5] for i in range(len(output))]}).set_index(output.index)
        amenity_seg_index=pd.DataFrame({"indices":[output.values[i][6] for i in range(len(output))]}).set_index(output.index)
        gsa_seg_index=pd.DataFrame({"indices":[output.values[i][7] for i in range(len(output))]}).set_index(output.index)
        gss_seg_index=pd.DataFrame({"indices":[output.values[i][8] for i in range(len(output))]}).set_index(output.index)
        

       
        
        socio_demo_index,result=affectation_bat(building,batiments_affectes,highways,result,socio_demo_index)
        loisire_seg_index,result=affectation_loisire(loisire,loisire_affectes,highways,result,loisire_seg_index)
        tourism_seg_index,result=affectation_tourisme(tourism,tourism_affectes,highways,result,tourism_seg_index)
        public_transport_seg_index,result= affectation_public_transport(public_transport,public_transport_affectes,highways,result,public_transport_seg_index)
        shop_seg_index,result=affectation_shops(shops,shop_affectes,highways,result,shop_seg_index)
        amenity_seg_index,result=affectation_amenity(amenity,amenity_affectes,highways,result,amenity_seg_index)
        gsa_seg_index,result=affectation_gsa(gsa,gsa_affectes,highways,result,gsa_seg_index)
        gss_seg_index,result=affectation_gss(gss,gss_affectes,highways,result,gss_seg_index)
        
        if final_result is None:
            final_result=result
        else:
            final_result=pd.concat([final_result,result])
        pickle.dump(result,open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\finale data\data\\"+place_name.lower()+".pkl","wb"))
        pickle.dump([
            socio_demo_index,
            loisire_seg_index,
            tourism_seg_index,
            public_transport_seg_index,
            shop_seg_index,
            amenity_seg_index,
            gsa_seg_index,
            gss_seg_index
            
        ],open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\finale data\indices\\"+place_name.lower()+".pkl","wb"))

Nord


C:\Users\Diametrix\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\285490972.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=loisire_non_aff.groupby(["seg","attracteur"]).size().reset_index().pivot("seg",'attracteur',0).fillna(0)
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\1227216181.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=shop_non_aff.groupby(["seg","secteur"]).size().reset_index().pivot("seg",'secteur',0).fillna(0)
C:\Users\Diametrix\AppData\Loca

Oise


C:\Users\Diametrix\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\285490972.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=loisire_non_aff.groupby(["seg","attracteur"]).size().reset_index().pivot("seg",'attracteur',0).fillna(0)
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\1227216181.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=shop_non_aff.groupby(["seg","secteur"]).size().reset_index().pivot("seg",'secteur',0).fillna(0)
C:\Users\Diametrix\AppData\Loca

Orne
error
Pas-de-Calais


...

Val-de-Marne


C:\Users\Diametrix\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\285490972.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=loisire_non_aff.groupby(["seg","attracteur"]).size().reset_index().pivot("seg",'attracteur',0).fillna(0)
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\1227216181.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=shop_non_aff.groupby(["seg","secteur"]).size().reset_index().pivot("seg",'secteur',0).fillna(0)
C:\Users\Diametrix\AppData\Loca

Val-D'Oise


C:\Users\Diametrix\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\285490972.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=loisire_non_aff.groupby(["seg","attracteur"]).size().reset_index().pivot("seg",'attracteur',0).fillna(0)
C:\Users\Diametrix\AppData\Local\Temp\ipykernel_3668\1227216181.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=shop_non_aff.groupby(["seg","secteur"]).size().reset_index().pivot("seg",'secteur',0).fillna(0)
C:\Users\Diametrix\AppData\Loca

Guadeloupe
error
Martinique
error
Guyane
error
La Réunion
error
Mayotte
error


# Vérification des résultats

In [110]:
cols_display=['P17_POP',"Attracteur_tourisme" , "metro",  "tram","transport","GSA",  "Sport",
             
             "Equipement maison",  "Equipement Personne",
              "Culture Loisir" , "Culture loisir -loisire",  

                        "Sport -loisire" , "Alimentaire" , "Mobilité",  

                       'Soin de la personne et Médical -amenity' , 

                        "pt d'intérert",  "Medical",  "restauration" , "Soin Personne" ,  

                        "Service public"  ,"tous -amenity"  ,

                        "Culture loisir -amenity",  


                        "Attracteur pour tous -loisire",  "Surface_de_vente_GSA"  
              
             ]

In [142]:
import os 
######################################################
#random departement 


deps=os.listdir(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\finale data\data")
row=deps[np.random.randint(0,len(deps))]

place_name=row.split(".")[0]
place_name="paris"
print(place_name)


res=pickle.load(open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\finale data\data\\"+place_name.lower()+".pkl","rb"))
indices=pickle.load(open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\finale data\indices\\"+place_name.lower()+".pkl","rb"))

socio_demo_index,loisire_seg_index,tourism_seg_index, public_transport_seg_index, shop_seg_index, amenity_seg_index, gsa_seg_index, gss_seg_indexindices=indices
building=pd.read_pickle(path+r"batiments_est_pop\\"+place_name+".pkl").to_crs('EPSG:3857')
            
highways=pd.read_pickle(path+r"rue\\"+place_name+"_highways.pkl").to_crs('EPSG:3857')

highways=highways.loc[~highways.index.duplicated(keep="first")]

shops=pd.read_pickle(path+r"magasin\\"+place_name+"_shops.pkl").to_crs('EPSG:3857')



try:
    gsa=pd.read_pickle(path+r"GSA\\"+place_name+"_gsa.pkl").to_crs('EPSG:3857')
except:
    gsa=None
try:
    gss=pd.read_pickle(path+r"GSS\\"+place_name+"_gss.pkl").to_crs('EPSG:3857')
except:
    gss=None


loisire=pd.read_pickle(path+r"loisire\\"+place_name+"_leisure.pkl").to_crs('EPSG:3857')
tourism=pd.read_pickle(path+r"tourisme\\"+place_name+"_tourism.pkl").to_crs('EPSG:3857')
public_transport=pd.read_pickle(path+r"public_transports\\"+place_name+".pkl").to_crs('EPSG:3857')
amenity=pd.read_pickle(path+r"amenity\\"+place_name+"_amenity.pkl").to_crs('EPSG:3857')

paris


In [220]:
i=np.random.choice(res.index)
i=("way", 68881723)
map_=highways.loc[[i]].explore(color="red")
map_=building.loc[socio_demo_index.loc[i].indices].explore(m=map_)
if len(shops.loc[shop_seg_index.loc[i].indices])>0:
    map_=shops.loc[shop_seg_index.loc[i].indices].explore(m=map_,color="green",marker_kwds={"radius":4})
    
if len(tourism.loc[tourism_seg_index.loc[i].indices])>0:
    map_=tourism.loc[tourism_seg_index.loc[i].indices].explore(m=map_,color="black",marker_kwds={"radius":4})
    
if len(loisire.loc[loisire_seg_index.loc[i].indices])>0:
    map_=loisire.loc[loisire_seg_index.loc[i].indices].explore(m=map_,color="purple",marker_kwds={"radius":4})
if len(public_transport.loc[public_transport_seg_index.loc[i].indices])>0:
    map_=public_transport.loc[public_transport_seg_index.loc[i].indices].explore(m=map_,color="yellow",marker_kwds={"radius":4})

if len(amenity.loc[amenity_seg_index.loc[i].indices])>0:
    map_=amenity.loc[amenity_seg_index.loc[i].indices].explore(m=map_,color="gray",marker_kwds={"radius":4})
try:
    if len(gsa.loc[gsa_seg_index.loc[i].indices])>0:
        map_=gsa.loc[gsa_seg_index.loc[i].indices].explore(m=map_,color="brown",marker_kwds={"radius":4})
        
except:
    pass
try:
    if len(gss.loc[gss_seg_index.loc[i].indices])>0:
     
        map_=gss.loc[gss_seg_index.loc[i].indices].explore(m=map_,color="orange",marker_kwds={"radius":4})
except:
    pass
res.loc[[i],cols_display]

,,P17_POP,Attracteur_tourisme,metro,tram,transport,GSA,Sport,Equipement maison,Equipement Personne,Culture Loisir,Culture loisir -loisire,Sport -loisire,Alimentaire,Mobilité,Soin de la personne et Médical -amenity,pt d'intérert,Medical,restauration,Soin Personne,Service public,tous -amenity,Culture loisir -amenity,Attracteur pour tous -loisire,Surface_de_vente_GSA
element_type,osmid,,,,,,,,,,,,,,,,,,,,,,,,
way,68881723,3813.072911,10,2,0,1,0.0,0.0,6.0,3.0,14.0,4.0,2.0,11.0,26.0,4.0,4.0,1.0,63.0,9.0,5.0,2.0,6.0,0.0,0.0


In [221]:
map_